# REST API

Create your own REST API using fastapi and uvicorn so you can interact with MongoDB over HTTP requests

##### Install Dependencies 

```bash
pip install fastapi
pip install "uvicorn[standard]"
```

<hr>

##### Put the following in a file called main.py

```python
from bson import ObjectId
from pymongo import MongoClient
from fastapi import FastAPI

### Setup MongoDB Client
mongodb_uri = "mongodb://localhost:27017/"
client = MongoClient(mongodb_uri)

### Setup FastAPI App
app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello World"}
```

<hr>

##### Run the uvicorn server in the directory of your main.py file

```bash
cd path/to/main_py_dir/
uvicorn main:app --reload
```

## Intro to 'requests' Python Module

In [1]:
import requests

In [2]:
x = requests.get('https://w3schools.com/python/demopage.htm')
print(x.text)

<!DOCTYPE html>
<html>
<body>

<h1>This is a Test Page</h1>

</body>
</html>


In [3]:
x = requests.get("http://localhost:8000/")
print(x.status_code)
print(x.text)

200
{"message":"Hello World"}


In [4]:
print(type(x.text))

<class 'str'>


In [5]:
print(x.json())
print(type(x.json()))

{'message': 'Hello World'}
<class 'dict'>


<hr>

## GET (find)

##### Add the following to main.py (and save the file)

```python
@app.get("/zips/{zip_id}")
async def get_zip_by_id(zip_id):
    return client.performance_db.zips.find_one({"_id": zip_id})
```
<hr>

In [6]:
x = requests.get("http://127.0.0.1:8000/zips/96819")
print(x.status_code)
print(x.text)

200
{"_id":"96819","city":"HONOLULU","loc":[-157.875947,21.34877],"pop":50584,"state":"HI"}


<hr>

##### Add the following to main.py (and save the file)

```python
@app.get("/zips/city/{city}")
async def get_zip_by_city(city):
    return list(client.performance_db.zips.find({"city": city}))
```
<hr>

In [7]:
x = requests.get("http://127.0.0.1:8000/zips/city/HONOLULU")
print(x.status_code)
print(x.text)

200
[{"_id":"96813","city":"HONOLULU","loc":[-157.852072,21.317905],"pop":23082,"state":"HI"},{"_id":"96818","city":"HONOLULU","loc":[-157.926925,21.353173],"pop":62915,"state":"HI"},{"_id":"96819","city":"HONOLULU","loc":[-157.875947,21.34877],"pop":50584,"state":"HI"},{"_id":"96821","city":"HONOLULU","loc":[-157.755242,21.292811],"pop":18366,"state":"HI"},{"_id":"96822","city":"HONOLULU","loc":[-157.829819,21.311704],"pop":39632,"state":"HI"},{"_id":"96826","city":"HONOLULU","loc":[-157.828388,21.294139],"pop":33672,"state":"HI"},{"_id":"96825","city":"HONOLULU","loc":[-157.698523,21.298684],"pop":27432,"state":"HI"},{"_id":"96815","city":"HONOLULU","loc":[-157.826616,21.281084],"pop":28650,"state":"HI"},{"_id":"96817","city":"HONOLULU","loc":[-157.861469,21.329452],"pop":48920,"state":"HI"},{"_id":"96814","city":"HONOLULU","loc":[-157.843876,21.299846],"pop":14182,"state":"HI"},{"_id":"96816","city":"HONOLULU","loc":[-157.800626,21.288677],"pop":49208,"state":"HI"}]


<hr>

## POST (insert)

##### Add the following to main.py (and save the file)

```python
@app.post("/users", status_code=201)
async def add_user(user_data: dict):
    insert_res = client.my_store.users.insert_one(user_data)
    return {"inserted_id": str(insert_res.inserted_id)}
```
<hr>

In [8]:
x = requests.post("http://localhost:8000/users/", json={"name": "REST User", "favorite_color": "red"})
print(x.status_code)
print(x.text)

201
{"inserted_id":"6205b301ca593d8bfac3de18"}


In [10]:
x = requests.get("http://localhost:8000/users/")
print(x.status_code)
print(x.text)

405
{"detail":"Method Not Allowed"}


<hr>

##### Add the following to main.py (and save the file)

```python
@app.get("/users")
async def get_user(user_data: dict):
    user_doc = client.my_store.users.find_one(user_data)
    user_doc["_id"] = str(user_doc["_id"])
    return user_doc 
```
<hr>

In [11]:
x = requests.get("http://localhost:8000/users/", json={"name": "REST User"})
print(x.status_code)
print(x.text)

200
{"_id":"6205b301ca593d8bfac3de18","name":"REST User","favorite_color":"red"}


In [13]:
user_id = x.json()["_id"]
print(user_id)

6205b301ca593d8bfac3de18


<hr>

## PUT (replace)

##### Add the following to main.py (and save the file)

```python
@app.put("/users/{user_id}")
async def replace_user(user_id, user_data: dict):
    replace_res = client.my_store.users.replace_one({"_id": ObjectId(str(user_id))}, user_data)
    return replace_res.raw_result
```
<hr>

In [14]:
x = requests.put("http://localhost:8000/users/" + user_id, json={"name": "Replaced REST User", "favorite_food": "eggs"})
print(x.status_code)
print(x.text)

200
{"n":1,"nModified":1,"ok":1.0,"updatedExisting":true}


<hr>

## PATCH (update)

##### Add the following to main.py (and save the file)

```python
@app.patch("/users/{user_id}")
async def update_user(user_id, user_data: dict):
    update_res = client.my_store.users.update_one({"_id": ObjectId(str(user_id))}, {"$set": user_data})
    return update_res.raw_result
```
<hr>

In [15]:
x = requests.patch("http://localhost:8000/users/" + user_id, json={"name": "Updated REST User"})
print(x.status_code)
print(x.text)

200
{"n":1,"nModified":1,"ok":1.0,"updatedExisting":true}


<hr>

## DELETE 

##### Add the following to main.py (and save the file)

```python
@app.delete("/users/{user_id}")
async def delete_user(user_id):
    delete_res = client.my_store.users.delete_one({"_id": ObjectId(str(user_id))})
    return delete_res.raw_result
```
<hr>

In [17]:
x = requests.delete("http://localhost:8000/users/" + user_id)
print(x.status_code)
print(x.text)

200
{"n":1,"ok":1.0}
